In [1]:
# === Adaptado para usar arquivos .yaml ===

from src.extract_text_from_docx import extract_text_from_docx
from src.extract_text_from_pdf import extract_text_from_pdf
from src.carregar_yaml import carregar_yaml

from crewai import Agent, Task, Crew, LLM
import torch

print("GPU disponível:", torch.cuda.is_available())

# LLM do Ollama
llm = LLM(
    model="ollama/mistral",
    base_url="http://localhost:11434"
)

# === Extrair o texto do currículo ===
def extract_text_from_resume(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    else:
        return "Unsupported file format."

res2 = extract_text_from_resume('files/Anita-Sanjok.docx')

# === Carregar Agente 1: Resume Feedback ===
agente1_cfg = carregar_yaml("agents/resume_feedback.yaml")
agente1_data = agente1_cfg["resume_feedback"]

resume_feedback = Agent(
    role=agente1_data["role"],
    goal=agente1_data["goal"],
    backstory=agente1_data["backstory"],
    llm=llm,
    verbose=True
)

task1_cfg = carregar_yaml("agents_tasks/resume_feedback_task.yaml")
tarefa1_data = task1_cfg["resume_feedback_task"]

resume_feedback_task = Task(
    description=tarefa1_data["description"],
    expected_output=tarefa1_data["expected_output"],
    agent=resume_feedback
)

# === Carregar Agente 2: Resume Advisor ===
agente2_cfg = carregar_yaml("agents/resume_advisor.yaml")
agente2_data = agente2_cfg["resume_advisor"]

resume_advisor = Agent(
    role=agente2_data["role"],
    goal=agente2_data["goal"],
    backstory=agente2_data["backstory"],
    llm=llm,
    verbose=True
)

task2_cfg = carregar_yaml("agents_tasks/resume_advisor_task.yaml")
tarefa2_data = task2_cfg["resume_advisor_task"]

# descricao2 = tarefa2_data["description"]

resume_advisor_task = Task(
    description=tarefa2_data["description"],
    expected_output=tarefa2_data["expected_output"],
    context=[resume_feedback_task],  # contexto vem da primeira tarefa
    agent=resume_advisor
)

# === Criar Crew e executar ===
crew = Crew(
    agents=[resume_feedback, resume_advisor],
    tasks=[resume_feedback_task, resume_advisor_task],
    verbose=False
)

result1 = crew.kickoff(inputs={"resume": res2})

GPU disponível: True
# Agent: Professional Resume Advisor
## Task: Give feedback on the resume to make it stand out for recruiters.  Review every section, inlcuding the summary, work experience, skills, and education. Suggest to add relevant sections if they are missing.   Also give an overall score to the resume out of 10.   This is the resume: Anita Sanjok                                           

Dwarka, 110078| 123-456-7890 | anitaname@delhiuni.edu | linkedin.com/in/anita

EDUCATION
University of Delhi, Delhi 								               May 2025
Master of Arts in Education 

Amity University, Jaipur									   May 2018
Bachelor of Arts in Sociology

RELEVANT PROFESSIONAL EXPERIENCE

University of Delhi, Vishwavidyalaya, Delhi	               		                           January  2024 – Present
Youth Development Specialist/Research Assistant
Facilitate activities for groups of 20-25 students ages 14-18 to develop self-efficacy and confidence in 2-3 north campus colleges 
Collaborate wit

In [2]:
# === Visualizar Resultados ===
from IPython.display import Markdown, display

markdown_content1 = resume_feedback_task.output.raw.strip("```markdown").strip("```").strip()
display(Markdown("## Feedback gerado pelo primeiro agente:\n" + markdown_content1))

## Feedback gerado pelo primeiro agente:
The overall score of the resume is 9 out of 10. Here's my feedback:

- The resume is well-structured and easy to read, with clear sections for education, experience, skills, and contact information. However, it would be beneficial to add a brief summary or objective statement at the beginning of the resume to give an overview of Anita's career goals and qualifications.

- In the education section, listing the expected graduation date for the Master of Arts in Education might help recruiters understand when Anita will be available for full-time work.

- The relevant professional experience section is strong and clearly demonstrates Anita's skills and achievements in her field. It would be helpful to quantify some of her accomplishments, such as the number of students she has worked with or the percentage increase in social media engagement.

- In the skills section, it would be beneficial to list any specific software or tools relevant to the job Anita is applying for, in addition to Microsoft Office Suite, SPSS, Qualtrics, and Canva. For example, if Anita has experience with Adobe Creative Suite, she should include that on her resume.

- Adding a section for certifications or awards could help make the resume stand out. If Anita has any relevant certifications or awards, she should list them in this section.

- It would be beneficial to tailor the resume to each specific job application by highlighting relevant skills and experiences for that position. This will show recruiters that Anita is a strong fit for their organization and increase her chances of getting an interview.

Overall, the resume is well-written and demonstrates Anita's strong qualifications and experience in her field. With some small tweaks, it has the potential to stand out in the job market and help Anita secure the position she wants.

In [3]:
markdown_content2 = resume_advisor_task.output.raw.strip("```markdown").strip("```").strip()
display(Markdown("## Currículo reescrito pelo segundo agente:\n" + markdown_content2))

## Currículo reescrito pelo segundo agente:
Anita Sanjok
Dwarka, 110078 | 123-456-7890 | [anitaname@delhiuni.edu](mailto:anitaname@delhiuni.edu) | linkedin.com/in/anita

Summary:
Motivated and dedicated Youth Development Specialist with a Master's degree in Education and extensive experience in facilitating workshops, conducting research, and engaging with students and parents on college and career preparation topics. Skilled in Microsoft Office Suite (Word, Excel, PowerPoint), SPSS, Qualtrics, Canva, and fluent in Spanish and English.

Education:
- University of Delhi, Delhi (Expected Graduation: May 2025) | Master of Arts in Education
- Amity University, Jaipur (Graduated: May 2018) | Bachelor of Arts in Sociology

Relevant Professional Experience:
**Youth Development Specialist/Research Assistant** - University of Delhi, Vishwavidyalaya, Delhi (January 2024 – Present)
- Facilitate activities for groups of 20-25 students ages 14-18 to develop self-efficacy and confidence in 2-3 north campus colleges
- Collaborate with program director and 5 specialists to gather quantitative and qualitative data, conduct literature reviews, and perform assessments to evaluate program effectiveness
- Code qualitative data from student insights in SPSS and determine themes in student experiences and reported findings to program director
- Plan activities, share insights, and discuss challenges with program staff during weekly meetings

**Program Coordinator** - Amity University, Jaipur (August 2021 – May 2024)
- Engaged with 20-25 high school students in groups to discuss college and career preparation topics such as major and career exploration, college applications, and scholarship research
- Hosted workshops for 40-50 parents to educate them about the college application process and financial resources to support higher education
- Partnered with schools administrators to coordinate logistics to implement workshops
- Collected survey feedback via Google Form to assess student and parent needs and satisfaction with workshops

**Program Coordinator, Office of Research and Sponsored Programs** - Amity University, Jaipur (June 2018 – July 2021)
- Collaborated with program directors and faculty to implement events, including orientation week activities, reaching 25-50 students per event
- Managed budget for student activities using Microsoft Excel and generated monthly budget reports for program directors
- Created and analyzed student experience surveys using Qualtrics and communicated key trends to program directors each quarter
- Designed graphics using Canva to promote co-curricular activities through email campaigns, achieving an open rate of 75%

Additional Work Experience:
**Program Assistant, Department of Sociology** - Amity University, Jaipur (August 2016 – May 2018)
- Liaised with academic advisors and Career Services to plan and implement career workshops and alumni and employer panels to expose students to career paths for Sociology majors
- Developed marketing materials including flyers and social media posts using Canva to promote events, increasing social media engagement by 20%

Skills:
- Proficient in Microsoft Office Suite (Word, Excel, PowerPoint), SPSS, Qualtrics, Canva
- Bilingual in Spanish and English

Certifications and Awards:
- [If any]